# 🕵️ Validador de Planos de Aula

Este notebook tem como objetivo analisar todos os arquivos de planejamento (`.txt`) gerados na pasta `aulas/`. Ele verifica se os campos obrigatórios, como `[CONTEUDO]` e `[ESTRATEGIA]`, foram devidamente preenchidos, garantindo que nenhum plano de aula incompleto seja enviado para o robô de registro.

**Fluxo de trabalho:**
1. Execute este notebook para ver a lista de planos de aula pendentes.
2. Preencha os arquivos apontados como "incompletos".
3. Execute o notebook novamente para confirmar que tudo está OK.
4. Com tudo validado, execute `registrar_aulas.py` com segurança.

## 1. Carregamento e Análise dos Arquivos

In [ ]:
import os
import pandas as pd

def parse_plano_aula(caminho_arquivo):
    """Lê um arquivo de plano de aula e extrai os dados, validando o preenchimento."""
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        return None

    plano = {
        'data': '',
        'horario': '',
        'conteudo_ok': False,
        'estrategia_ok': False,
        'link_ok': False
    }

    # Extrai data e horário dos comentários
    for linha in content.splitlines():
        if linha.startswith('# Data:'):
            plano['data'] = linha.split(':', 1)[1].strip()
        elif linha.startswith('# Horário:'):
            plano['horario'] = linha.split(':', 1)[1].strip()

    # Valida o preenchimento dos blocos
    blocos = {
        '[CONTEUDO]': 'conteudo_ok',
        '[ESTRATEGIA]': 'estrategia_ok',
        '[RECURSO_LINK]': 'link_ok'
    }

    for bloco_str, chave_ok in blocos.items():
        try:
            start_index = content.index(bloco_str) + len(bloco_str)
            # Encontra o próximo bloco ou o fim do arquivo
            next_block_index = len(content)
            for next_b in blocos:
                try:
                    find_idx = content.index(next_b, start_index)
                    if find_idx < next_block_index:
                        next_block_index = find_idx
                except ValueError:
                    continue

            valor = content[start_index:next_block_index].strip()
            
            # A validação principal: o campo não está vazio e não contém o texto padrão
            if valor and 'Preencher' not in valor:
                plano[chave_ok] = True
            # Para o link, qualquer conteúdo é considerado OK
            elif chave_ok == 'link_ok' and valor:
                plano[chave_ok] = True

        except ValueError:
            continue # Bloco não encontrado
            
    return plano

# --- Script Principal ---

project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
aulas_dir = os.path.join(project_root, 'aulas')

planos_data = []

if os.path.exists(aulas_dir):
    for turma_folder in os.listdir(aulas_dir):
        turma_path = os.path.join(aulas_dir, turma_folder)
        if os.path.isdir(turma_path):
            for filename in os.listdir(turma_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(turma_path, filename)
                    parsed_data = parse_plano_aula(file_path)
                    
                    if parsed_data:
                        disciplina_curto = filename.split('_')[0]
                        planos_data.append({
                            'Turma': turma_folder.replace('_', 'º'),
                            'Disciplina': disciplina_curto,
                            'Data': parsed_data['data'],
                            'Horário': parsed_data['horario'],
                            'Conteúdo OK': '✅' if parsed_data['conteudo_ok'] else '❌',
                            'Estratégia OK': '✅' if parsed_data['estrategia_ok'] else '❌',
                            'Link OK': '✅' if parsed_data['link_ok'] else '➖',
                            'Arquivo': os.path.join(turma_folder, filename)
                        })

if planos_data:
    df_planos = pd.DataFrame(planos_data)
    df_planos.sort_values(by=['Turma', 'Data', 'Horário'], inplace=True)
else:
    df_planos = pd.DataFrame()

print(f"Total de arquivos de planejamento encontrados: {len(df_planos)}")

## 2. Relatório Geral de Planos

A tabela abaixo mostra todos os planos de aula encontrados e o status de preenchimento de cada campo.

In [ ]:
if not df_planos.empty:
    display(df_planos.style.set_properties(**{'text-align': 'left'}))
else:
    print("Nenhum plano de aula encontrado na pasta 'aulas/'. Execute 'preparar_planos.py' primeiro.")

## 3. Ação Necessária: Planos Incompletos

**Atenção!** Os arquivos listados abaixo precisam ser preenchidos antes de executar o robô de registro.

In [ ]:
if not df_planos.empty:
    df_incompletos = df_planos[
        (df_planos['Conteúdo OK'] == '❌') | 
        (df_planos['Estratégia OK'] == '❌')
    ]

    if not df_incompletos.empty:
        print(f"Encontrados {len(df_incompletos)} planos de aula incompletos:")
        display(df_incompletos[['Turma', 'Disciplina', 'Data', 'Arquivo']])
    else:
        print("🎉 Ótima notícia! Todos os planos de aula estão preenchidos e prontos para serem registrados.")